In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

from rtbm import RTBM, minimizer

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy


In [2]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [ ]:
# Visualize individual pics
i=10
print(Y_train[i])
I=np.reshape(X_train[i], (28,28))
plt.imshow(I, interpolation='nearest',  cmap='gray_r')
plt.show()

# Logistic regression baseline

In [ ]:
# Logistic regression baseline
logreg = linear_model.LogisticRegression(multi_class='multinomial',solver='lbfgs')

logreg.fit(X_train,Y_train)


In [ ]:
# On train set

P=logreg.predict(X_train)

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))


In [ ]:
# On test set
P=logreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


# Linear regression base line

In [ ]:
linreg = linear_model.LinearRegression()

linreg.fit(X_train,Y_train)


In [ ]:
# On train set

P=np.round(linreg.predict(X_train))

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))

In [ ]:
# On test set
P=linreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


# Linear regression via CMA

In [17]:
M = mdl.Model()
M.add(layers.Linear(784,1))

minim = minimizer.CMA(False)
sol=minim.train(mse, M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

CMA on 1 cpu(s) enabled
(11_w,23)-aCMA-ES (mu_w=6.7,w_1=25%) in dimension 785 (seed=115327, Tue Nov  7 06:26:28 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     23 2.684171781264688e+01 1.0e+00 9.91e-01  1e+00  1e+00 0:00.0
    2     46 2.899058970342607e+01 1.0e+00 9.82e-01  1e+00  1e+00 0:00.1
    3     69 3.108250571239321e+01 1.0e+00 9.74e-01  1e+00  1e+00 0:00.2
   28    644 4.249590425783825e+01 1.0e+00 8.52e-01  9e-01  9e-01 0:03.4
   60   1380 4.559654482485951e+01 1.0e+00 7.74e-01  8e-01  8e-01 0:07.4
  100   2300 4.337709372524052e+01 1.0e+00 7.23e-01  7e-01  7e-01 0:12.4
  148   3404 4.950557147307769e+01 1.1e+00 6.94e-01  7e-01  7e-01 0:18.5
  200   4600 5.024874889295326e+01 1.1e+00 6.79e-01  7e-01  7e-01 0:25.0
  264   6072 4.667633734614471e+01 1.1e+00 6.47e-01  6e-01  6e-01 0:33.0
  300   6900 4.656582592535251e+01 1.1e+00 6.31e-01  6e-01  6e-01 0:37.6
  379   8717 4.712995730118923e+01 1.2e+00 6.03e-01  6e-01  6e-01 0:47.6
  400  

KeyboardInterrupt: 

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

# Linear regression via SGD

In [47]:
M = mdl.Model()
M.add(layers.Linear(784,100))
M.add(layers.Linear(100,1))

#M.add(layers.NonLinear(784,3))
#M.add(layers.NonLinear(3,3))
#M.add(layers.Linear(10,1))

minim = minimizer.SGD()
sol=minim.train(mse, M, np.transpose(X_train), Y_train.reshape(1,len(Y_train)), lr=0.1, maxiter=1000, batch_size=1000)

#minim = minimizer.BFGS()
#sol=minim.train(mse(), M, np.transpose(X_train), Y_train.reshape(1,len(Y_train)), maxiter=1000)

Iteration 0 in 0.16(s), cost = 198.358939
Iteration 100 in 16.49(s), cost = 17.150658
Iteration 200 in 33.00(s), cost = 9.652766
Iteration 300 in 51.16(s), cost = 6.891805
Iteration 400 in 69.33(s), cost = 5.419768
Iteration 500 in 87.49(s), cost = 4.506081
Iteration 600 in 105.66(s), cost = 3.889812
Iteration 700 in 123.82(s), cost = 3.451282
Iteration 800 in 141.73(s), cost = 3.127251
Iteration 900 in 159.59(s), cost = 2.881055
('Cost: ', 2.691659388290869)
('Sol: ', array([-0.33338102,  0.33528365,  0.2477624 , ...,  0.17442351,
        0.19408339,  0.04494785]))
Time: 177 s


In [48]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.41      0.16      0.23      1001
        1.0       0.26      0.20      0.23      1127
        2.0       0.18      0.22      0.20       991
        3.0       0.16      0.20      0.18      1032
        4.0       0.11      0.15      0.13       980
        5.0       0.13      0.20      0.16       863
        6.0       0.16      0.20      0.18      1014
        7.0       0.21      0.20      0.20      1070
        8.0       0.15      0.11      0.13       944
        9.0       0.39      0.14      0.21       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0
       13.0       0.00      0.00      0.00         0
       14.0       0.00      0.00      0.00         0

avg / total       0.22      0.18      0.19     10000

[[164 309 219 155  74  50  16   9   5   0   0   0   0   0   0]
 [ 85 230 365 263 116  47  17   4

In [28]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))


             precision    recall  f1-score   support

        0.0       0.70      0.20      0.31       980
        1.0       0.34      0.26      0.30      1135
        2.0       0.18      0.21      0.19      1032
        3.0       0.22      0.28      0.25      1010
        4.0       0.11      0.15      0.13       982
        5.0       0.16      0.27      0.20       892
        6.0       0.14      0.22      0.18       958
        7.0       0.21      0.23      0.22      1028
        8.0       0.19      0.12      0.15       974
        9.0       0.45      0.10      0.16      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0

avg / total       0.27      0.21      0.21     10000

[[196 355 218 117  51  35   6   2   0   0   0   0]
 [  3 300 467 234  88  26  12   4   1   0   0   0]
 [ 56 147 220 248 206 111  36   6   0   2   0   0]
 [ 22  67 198 280 218 124  62  27  11   0   1   0]
 [  0   1   9  64 147 282 282 141  49   7   0   0]


# E(h|v) via SGD

In [14]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,1))
#M.add(layers.DiagExpectationUnitLayer(10,1))

minim = minimizer.SGD()
sol=minim.train(mse, M, np.transpose(X_train), Y_train.reshape(1,Y_train.shape[0]),lr=100,momentum=0.1, nesterov=True, maxiter=1000, batch_size=1000)

Iteration 0 in 0.50(s), cost = 4.009833


KeyboardInterrupt: 

Process PoolWorker-15:
Process PoolWorker-10:
Process PoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-11:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-12:
Process PoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
Process PoolWorker-14:
Process PoolWorker-9:
    self.run()
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/usr/li

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))



# E(h|v) via CMA

In [9]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,3, phase=1))
M.add(layers.DiagExpectationUnitLayer(3,1, phase=1))

minim = minimizer.CMA(False)
sol=minim.train(mse, M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

CMA on 1 cpu(s) enabled
(13_w,27)-aCMA-ES (mu_w=7.8,w_1=22%) in dimension 2363 (seed=1057215, Mon Nov  6 06:03:49 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     27 8.432775907943645e+00 1.0e+00 9.96e-01  1e+00  1e+00 0:00.2
    2     54 7.165652619614564e+00 1.0e+00 9.93e-01  1e+00  1e+00 0:01.5
    3     81 7.912105729280427e+00 1.0e+00 9.89e-01  1e+00  1e+00 0:02.7
    6    162 5.418252810319234e+00 1.0e+00 9.79e-01  1e+00  1e+00 0:06.1
   10    270 3.968259427223221e+00 1.0e+00 9.68e-01  1e+00  1e+00 0:10.8
   15    405 3.519140165301580e+00 1.0e+00 9.54e-01  1e+00  1e+00 0:15.9
   20    540 3.488005967874373e+00 1.0e+00 9.42e-01  9e-01  9e-01 0:22.2
   28    756 3.043410483177555e+00 1.0e+00 9.24e-01  9e-01  9e-01 0:30.2
   35    945 2.851399375488116e+00 1.0e+00 9.10e-01  9e-01  9e-01 0:38.2
   45   1215 2.675786713304412e+00 1.0e+00 8.94e-01  9e-01  9e-01 0:47.6
   55   1485 2.531320498799549e+00 1.0e+00 8.79e-01  9e-01  9e-01 0:58.3
   67

In [10]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.62      0.15      0.25      1001
        1.0       0.33      0.25      0.28      1127
        2.0       0.19      0.24      0.21       991
        3.0       0.20      0.27      0.23      1032
        4.0       0.11      0.17      0.13       980
        5.0       0.13      0.22      0.16       863
        6.0       0.14      0.18      0.16      1014
        7.0       0.25      0.24      0.25      1070
        8.0       0.20      0.12      0.15       944
        9.0       0.45      0.09      0.15       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0

avg / total       0.26      0.20      0.20     10000

[[155 305 254 145  73  44  16   8   1   0   0   0]
 [ 20 284 388 279 116  25  11   4   0   0   0   0]
 [ 51 154 241 244 170  83  30  14   3   1   0   0]
 [ 17  71 225 278 256 106  52  17   8   2   0   0]
 [  4  13  32  69 169 261 230 145  47  10   0   0]


In [11]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))


             precision    recall  f1-score   support

        0.0       0.54      0.14      0.22       980
        1.0       0.31      0.24      0.27      1135
        2.0       0.19      0.23      0.21      1032
        3.0       0.19      0.25      0.22      1010
        4.0       0.10      0.15      0.12       982
        5.0       0.15      0.24      0.18       892
        6.0       0.13      0.19      0.16       958
        7.0       0.21      0.22      0.22      1028
        8.0       0.23      0.14      0.17       974
        9.0       0.45      0.12      0.19      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0

avg / total       0.25      0.19      0.20     10000

[[137 297 274 143  71  40  14   3   1   0   0   0]
 [ 28 278 390 293 106  33   4   3   0   0   0   0]
 [ 62 172 235 213 180 106  49   9   5   1   0   0]
 [ 19  92 189 253 236 121  72  19   7   1   1   0]
 [  3   8  21  59 149 270 266 160  36   8   2   0]
